<a href="https://colab.research.google.com/github/Eva455989/Work/blob/master/%D0%92%D0%B0%D1%80%D0%BD%D0%B0%D0%B2%D0%B8%D0%BD%D0%B0_%D0%95%D0%95%2C_%D0%9F%D1%80%D0%BE%D0%BB%D0%BE%D0%BD%D0%B3%D0%B0%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Блок 1: Импорт библиотек и функции**

In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Функция для очистки числовых значений
def clean_numeric_value(value):
    if pd.isna(value) or value == '' or value == 'стоп' or value == 'в ноль':
        return 0
    if isinstance(value, str):
        # Удаляем пробелы, заменяем запятые на точки
        value = value.replace(' ', '').replace(',', '.')
        # Удаляем нечисловые символы кроме точек и минусов
        value = ''.join(char for char in value if char.isdigit() or char in '.-')
        if value == '' or value == '-':
            return 0
    try:
        return float(value)
    except:
        return 0

# Функция для нормализации названий месяцев
def normalize_month(month_str):
    if pd.isna(month_str):
        return None
    month_str = str(month_str).lower()
    for ru, en in month_translation.items():
        if ru in month_str:
            return en + month_str[len(ru):]
    return month_str

**Блок 2: Загрузка данных**

In [15]:
# Загрузка реальных данных
print("Загрузка данных...")
financial_data = pd.read_csv('financial_data.csv')
prolongations = pd.read_csv('prolongations.csv')

print(f"Финансовые данные: {financial_data.shape}")
print(f"Данные по пролонгациям: {prolongations.shape}")

Загрузка данных...
Финансовые данные: (451, 19)
Данные по пролонгациям: (477, 3)


**Блок 3: Предобработка данных**

In [16]:
# Предобработка данных
print("\nПредобработка данных...")

# Получаем список финансовых колонок (исключаем служебные)
financial_columns = [col for col in financial_data.columns
                    if col not in ['id', 'Причина дубля', 'Account']]

print(f"Финансовые колонки: {financial_columns}")

# Очищаем финансовые данные
for col in financial_columns:
    financial_data[col] = financial_data[col].apply(clean_numeric_value)

# Агрегируем финансовые данные по id (суммируем дубли)
financial_aggregated = financial_data.groupby(['id', 'Account'])[financial_columns].sum().reset_index()

print(f"После агрегации: {financial_aggregated.shape}")

# Создаем словарь для перевода месяцев
month_translation = {
    'январь': 'Январь', 'февраль': 'Февраль', 'март': 'Март', 'апрель': 'Апрель',
    'май': 'Май', 'июнь': 'Июнь', 'июль': 'Июль', 'август': 'Август',
    'сентябрь': 'Сентябрь', 'октябрь': 'Октябрь', 'ноябрь': 'Ноябрь', 'декабрь': 'Декабрь'
}

# Нормализуем месяцы в prolongations
prolongations['month_normalized'] = prolongations['month'].apply(normalize_month)

# Создаем список месяцев для анализа (2023 год)
months_2023 = [f"{month} 2023" for month in [
    'Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь',
    'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'
]]

print(f"Месяцы для анализа: {months_2023}")


Предобработка данных...
Финансовые колонки: ['Ноябрь 2022', 'Декабрь 2022', 'Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023', 'Ноябрь 2023', 'Декабрь 2023', 'Январь 2024', 'Февраль 2024']
После агрегации: (314, 18)
Месяцы для анализа: ['Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023', 'Ноябрь 2023', 'Декабрь 2023']


**Блок 4: Функция расчета коэффициентов**

In [17]:
# Основная функция расчета коэффициентов
def calculate_prolongation_coefficients(prolongations_df, financial_df, months):
    """
    Расчет коэффициентов пролонгации для каждого менеджера и месяца

    КОЭФФИЦИЕНТ 1 МЕСЯЦА
    Формула: Коэф1 = Σ(Отгрузка в месяце M проектов, завершившихся в M-1) /
                   Σ(Отгрузка в последнем месяце проектов, завершившихся в M-1)

    Пример для мая 2023:
    - Проекты завершились в: Апрель 2023
    - Отгрузка в последнем месяце: Апрель 2023 (знаменатель)
    - Отгрузка в текущем месяце: Май 2023 (числитель)

    КОЭФФИЦИЕНТ 2 МЕСЯЦА
    Формула: Коэф2 = Σ(Отгрузка в месяце M проектов из M-2 БЕЗ отгрузки в M-1) /
                   Σ(Отгрузка в последнем месяце проектов из M-2 БЕЗ отгрузки в M-1)

    Пример для мая 2023:
    - Проекты завершились в: Март 2023
    - У которых НЕТ отгрузки в: Апрель 2023
    - Отгрузка в последнем месяце: Март 2023 (знаменатель)
    - Отгрузка в текущем месяце: Май 2023 (числитель)

    ОСОБЫЕ СЛУЧАИ:
    - Если знаменатель = 0, коэффициент = 0
    - Проекты учитываются только если они официально завершены (есть в prolongations)
    - Учитываются только проекты конкретного менеджера
    """
    results = []

    for i, current_month in enumerate(months):
        if i < 1:  # Для первого месяца нет данных о предыдущих завершениях
            continue

        prev_month = months[i-1]  # Месяц завершения проектов для коэффициента 1
        prev_prev_month = months[i-2] if i >= 2 else None  # Месяц завершения для коэффициента 2

        print(f"Обрабатываем {current_month}: предыдущий {prev_month}, предпредыдущий {prev_prev_month}")

        # Получаем всех менеджеров
        managers = prolongations_df['AM'].unique()

        for manager in managers:
            # КОЭФФИЦИЕНТ 1: Пролонгация в первый месяц
            projects_ended_prev_month = prolongations_df[
                (prolongations_df['month_normalized'] == prev_month) &
                (prolongations_df['AM'] == manager)
            ]['id'].unique()

            coef1 = 0
            if len(projects_ended_prev_month) > 0:
                last_month_shipment = 0
                current_month_shipment = 0

                for project_id in projects_ended_prev_month:
                    project_data = financial_df[financial_df['id'] == project_id]
                    if len(project_data) > 0:
                        # Сумма в последнем месяце реализации
                        if prev_month in financial_df.columns:
                            last_month_shipment += project_data[prev_month].iloc[0]

                        # Сумма в текущем месяце (пролонгация)
                        if current_month in financial_df.columns:
                            current_month_shipment += project_data[current_month].iloc[0]

                # Расчет коэффициента 1
                if last_month_shipment > 0:
                    coef1 = current_month_shipment / last_month_shipment

            # КОЭФФИЦИЕНТ 2: Пролонгация во второй месяц
            coef2 = 0
            if prev_prev_month:
                projects_ended_prev_prev_month = prolongations_df[
                    (prolongations_df['month_normalized'] == prev_prev_month) &
                    (prolongations_df['AM'] == manager)
                ]['id'].unique()

                if len(projects_ended_prev_prev_month) > 0:
                    total_last_month_shipment = 0
                    current_month_shipment_coef2 = 0

                    for project_id in projects_ended_prev_prev_month:
                        project_data = financial_df[financial_df['id'] == project_id]
                        if len(project_data) > 0:
                            # Проверяем, была ли отгрузка в предыдущем месяце
                            shipment_prev_month = project_data[prev_month].iloc[0] if prev_month in financial_df.columns else 0

                            if shipment_prev_month == 0:  # Не было пролонгации в первый месяц
                                # Сумма в последнем месяце реализации
                                if prev_prev_month in financial_df.columns:
                                    total_last_month_shipment += project_data[prev_prev_month].iloc[0]

                                # Сумма в текущем месяце (пролонгация во второй месяц)
                                if current_month in financial_df.columns:
                                    current_month_shipment_coef2 += project_data[current_month].iloc[0]

                    # Расчет коэффициента 2
                    if total_last_month_shipment > 0:
                        coef2 = current_month_shipment_coef2 / total_last_month_shipment

            results.append({
                'Месяц': current_month,
                'Менеджер': manager,
                'Коэффициент пролонгации 1 месяц': round(coef1, 4),
                'Коэффициент пролонгации 2 месяц': round(coef2, 4),
                'Количество проектов завершившихся в предыдущем месяце': len(projects_ended_prev_month)
            })

    return pd.DataFrame(results)

**Блок 5: Выполнение расчета**

In [18]:
# Выполняем расчет
print("\nНачинаем расчет коэффициентов пролонгации...")
results_df = calculate_prolongation_coefficients(prolongations, financial_aggregated, months_2023)

# Добавляем общие коэффициенты по отделу
department_monthly = results_df.groupby('Месяц').agg({
    'Коэффициент пролонгации 1 месяц': 'mean',
    'Коэффициент пролонгации 2 месяц': 'mean',
    'Количество проектов завершившихся в предыдущем месяце': 'sum'
}).reset_index()
department_monthly['Менеджер'] = 'Все отдел'

# Объединяем результаты
final_results = pd.concat([results_df, department_monthly], ignore_index=True)

# Расчет годовых коэффициентов
yearly_coefficients = final_results[final_results['Менеджер'] != 'Все отдел'].groupby('Менеджер').agg({
    'Коэффициент пролонгации 1 месяц': 'mean',
    'Коэффициент пролонгации 2 месяц': 'mean'
}).reset_index()
yearly_coefficients['Месяц'] = '2023 год'

# Общие годовые коэффициенты по отделу
department_yearly = pd.DataFrame({
    'Менеджер': ['Все отдел'],
    'Коэффициент пролонгации 1 месяц': [yearly_coefficients['Коэффициент пролонгации 1 месяц'].mean()],
    'Коэффициент пролонгации 2 месяц': [yearly_coefficients['Коэффициент пролонгации 2 месяц'].mean()],
    'Месяц': ['2023 год'],
    'Количество проектов завершившихся в предыдущем месяце': [0]
})

# Финальный датафрейм со всеми результатами
final_with_yearly = pd.concat([final_results, yearly_coefficients, department_yearly], ignore_index=True)

print(f"Всего записей: {len(final_with_yearly)}")
print(f"Уникальных менеджеров: {final_with_yearly['Менеджер'].nunique()}")

# Сохраняем основные результаты
final_with_yearly.to_csv('prolongation_coefficients.csv', index=False, encoding='utf-8-sig')
print("Основные результаты сохранены в 'prolongation_coefficients.csv'")


Начинаем расчет коэффициентов пролонгации...
Обрабатываем Февраль 2023: предыдущий Январь 2023, предпредыдущий None
Обрабатываем Март 2023: предыдущий Февраль 2023, предпредыдущий Январь 2023
Обрабатываем Апрель 2023: предыдущий Март 2023, предпредыдущий Февраль 2023
Обрабатываем Май 2023: предыдущий Апрель 2023, предпредыдущий Март 2023
Обрабатываем Июнь 2023: предыдущий Май 2023, предпредыдущий Апрель 2023
Обрабатываем Июль 2023: предыдущий Июнь 2023, предпредыдущий Май 2023
Обрабатываем Август 2023: предыдущий Июль 2023, предпредыдущий Июнь 2023
Обрабатываем Сентябрь 2023: предыдущий Август 2023, предпредыдущий Июль 2023
Обрабатываем Октябрь 2023: предыдущий Сентябрь 2023, предпредыдущий Август 2023
Обрабатываем Ноябрь 2023: предыдущий Октябрь 2023, предпредыдущий Сентябрь 2023
Обрабатываем Декабрь 2023: предыдущий Ноябрь 2023, предпредыдущий Октябрь 2023

Расчет завершен!
Всего записей: 132
Уникальных менеджеров: 11
Основные результаты сохранены в 'prolongation_coefficients.csv'


**Блок 6: Создание отчета и вывод статистики**

In [19]:
# Создаем расширенный аналитический отчет
print("\nСоздание аналитического отчета...")

with pd.ExcelWriter('prolongation_analysis_report.xlsx', engine='openpyxl') as writer:

    # Лист 1: Сводные коэффициенты по месяцам
    summary_1m = final_results.pivot_table(
        index='Менеджер',
        columns='Месяц',
        values='Коэффициент пролонгации 1 месяц',
        aggfunc='mean'
    ).round(4)

    summary_2m = final_results.pivot_table(
        index='Менеджер',
        columns='Месяц',
        values='Коэффициент пролонгации 2 месяц',
        aggfunc='mean'
    ).round(4)

    # Объединяем оба коэффициента в один датафрейм
    summary_combined = pd.concat([summary_1m, summary_2m], keys=['Коэф. 1 месяц', 'Коэф. 2 месяц'], axis=1)
    summary_combined.to_excel(writer, sheet_name='Сводные коэффициенты')

    # Лист 2: Годовые результаты
    yearly_data = final_with_yearly[final_with_yearly['Месяц'] == '2023 год']
    yearly_data = yearly_data[['Менеджер', 'Коэффициент пролонгации 1 месяц', 'Коэффициент пролонгации 2 месяц']]
    yearly_data.to_excel(writer, sheet_name='Годовые результаты', index=False)

    # Лист 3: Детализация по месяцам
    monthly_detail = final_with_yearly[final_with_yearly['Месяц'] != '2023 год']
    monthly_detail = monthly_detail.sort_values(['Месяц', 'Менеджер'])
    monthly_detail.to_excel(writer, sheet_name='Детализация по месяцам', index=False)

    # Лист 4: Рейтинг менеджеров
    manager_stats = prolongations.groupby('AM').agg({
        'id': 'count',
        'month_normalized': 'nunique'
    }).reset_index()
    manager_stats.columns = ['Менеджер', 'Всего проектов', 'Месяцев с завершениями']

    # Добавляем годовые коэффициенты
    ranking_data = pd.merge(manager_stats, yearly_data[yearly_data['Менеджер'] != 'Все отдел'], on='Менеджер')
    ranking_data['Общий рейтинг'] = (
        ranking_data['Коэффициент пролонгации 1 месяц'] * 0.7 +
        ranking_data['Коэффициент пролонгации 2 месяц'] * 0.3
    )
    ranking_data = ranking_data.sort_values('Общий рейтинг', ascending=False)
    ranking_data['Место'] = range(1, len(ranking_data) + 1)

    ranking_data.to_excel(writer, sheet_name='Рейтинг менеджеров', index=False)

    # Лист 5: Статистика проектов
    project_stats = prolongations['month_normalized'].value_counts().reset_index()
    project_stats.columns = ['Месяц завершения', 'Количество проектов']
    project_stats = project_stats.sort_values('Месяц завершения')
    project_stats.to_excel(writer, sheet_name='Статистика проектов', index=False)

print("Аналитический отчет создан: 'prolongation_analysis_report.xlsx'")

# Выводим краткую статистику
print("КРАТКАЯ СТАТИСТИКА")
print(f"Всего менеджеров: {len(prolongations['AM'].unique())}")
print(f"Всего проектов: {len(prolongations['id'].unique())}")
print(f"Период анализа: {months_2023[0]} - {months_2023[-1]}")

# Годовые результаты по отделу
dept_year = final_with_yearly[
    (final_with_yearly['Менеджер'] == 'Все отдел') &
    (final_with_yearly['Месяц'] == '2023 год')
]
if len(dept_year) > 0:
    print(f"\nРезультаты отдела за 2023 год:")
    print(f"Коэффициент пролонгации 1 месяц: {dept_year['Коэффициент пролонгации 1 месяц'].iloc[0]:.2%}")
    print(f"Коэффициент пролонгации 2 месяц: {dept_year['Коэффициент пролонгации 2 месяц'].iloc[0]:.2%}")

# Лучший менеджер
if 'ranking_data' in locals() and len(ranking_data) > 0:
    best_manager = ranking_data.iloc[0]
    print(f"\nЛучший менеджер: {best_manager['Менеджер']}")
    print(f"Общий рейтинг: {best_manager['Общий рейтинг']:.4f}")
    print(f"Коэффициент 1 месяц: {best_manager['Коэффициент пролонгации 1 месяц']:.2%}")
    print(f"Коэффициент 2 месяц: {best_manager['Коэффициент пролонгации 2 месяц']:.2%}")


Создание аналитического отчета...
Аналитический отчет создан: 'prolongation_analysis_report.xlsx'

=== КРАТКАЯ СТАТИСТИКА ===
Всего менеджеров: 10
Всего проектов: 313
Период анализа: Январь 2023 - Декабрь 2023

Результаты отдела за 2023 год:
Коэффициент пролонгации 1 месяц: 33.25%
Коэффициент пролонгации 2 месяц: 5.96%

Лучший менеджер: Михайлов Андрей Сергеевич
Общий рейтинг: 0.4626
Коэффициент 1 месяц: 66.08%
Коэффициент 2 месяц: 0.00%
